# Planning Graph Search

In [1]:
import copy
import numpy

In [2]:
initial_state = []

In [3]:
class In:
    def __init__(self, obj, place, negation = False):
        self.object = obj
        self.place = place
        self.negation = negation
        
    def __hash__(self):
        return hash((self.object, self.place, self.negation))
        
    def __eq__(self, other):
        return isinstance(other, In) and self.object == other.object and self.place == other.place and self.negation == other.negation
    
    def __invert__(self):
        if self.negation:
             return In(self.object, self.place, negation = False)
        else:
            return In(self.object, self.place, negation = True)
        
    
       
    def __str__(self):
        if self.negation:
            return f'~In({self.object}, {self.place})'
        else:   
            return f'In({self.object}, {self.place})'
        

class Holding:
    def __init__(self, obj, negation = False):
        self.object = obj
        self.negation = negation
        
    def __hash__(self):
        return hash((self.object, self.negation))
        
    def __eq__(self, other):
        return isinstance(other, Holding) and self.object == other.object and self.negation == other.negation
    
    def __invert__(self):
        if self.negation:
             return Holding(self.object, negation = False)
        else:
            return Holding(self.object, negation = True)
    
    def __str__(self):
        if self.negation:
            return f'~Holding({self.object})'
        else:
            return f'Holding({self.object})'
        
class Locked:
    def __init__(self, obj, negation = False):
        self.object = obj
        self.negation = negation
        
    def __hash__(self):
        return hash((self.object, self.negation))
        
    def __eq__(self, other):
        return isinstance(other, Locked) and self.object == other.object and self.negation == other.negation
    
    def __invert__(self):
        if self.negation:
             return Locked(self.object, negation = False)
        else:
            return Locked(self.object, negation = True)
        
    def __str__(self):
        if self.negation:
            return f'~Locked({self.object})'
        else:
            return f'Locked({self.object})'
    
        


In [4]:
initial_state.append(In('Robot', 'R2'))
initial_state.append(In('Key', 'R2'))
initial_state.append(Locked('Door', negation = True))

In [5]:
print(' ^ '.join([str(x) for x in initial_state]))

In(Robot, R2) ^ In(Key, R2) ^ ~Locked(Door)


In [6]:
class GraspKey:
    def __init__(self):
        self.preconditions = [In('Robot', 'R2'), In('Key', 'R2')]
        self.implications = [~In('Robot', 'R1'), ~In('Key', 'R1')]
        self.delete = []
        self.effect = [Holding('Key')]
    def action(self, state):
        state = copy.deepcopy(state)
        if all(precondition in state for precondition in self.preconditions):
            for element in self.delete:
                state.remove(element)
            for element in self.effect:
                state.append(element)
        return list(set(state))
    def __str__(self):
        return f'Grasp Key'
    
class LockDoor:
    def __init__(self):
        self.preconditions = [Holding('Key'), ~Locked('Door')]
        self.implications = []
        self.delete = [~Locked('Door')]
        self.effect = [Locked('Door')]
    def action(self, state):
        state = copy.deepcopy(state)
        if all(precondition in state for precondition in self.preconditions):
            for element in self.delete:
                state.remove(element)
            for element in self.effect:
                state.append(element)
        return list(set(state))
    def __str__(self):
        return f'Lock Door'
    
class PutKeyIntoBox:
    def __init__(self):
        self.preconditions = [In('Robot', 'R1'),Holding('Key'), In('Key', 'R1')]
        self.implications = [~In('Robot', 'R2'), ~In('Key', 'R2')]
        self.delete = [Holding('Key'), In('Key', 'R1')]
        self.effect = [In('Key', 'Box')]
    def action(self, state):
        state = copy.deepcopy(state)
        if all(precondition in state for precondition in self.preconditions):
            for element in self.delete:
                state.remove(element)
            for element in self.effect:
                state.append(element)
        return list(set(state))
    def __str__(self):
        return f'Put the key into the box'
    
class Move:
    def __init__(self, from_, to_):
        self.ubication = from_
        self.destiny = to_
        self.preconditions = [~Locked('Door'), In('Robot', self.ubication)]
        self.implications = [~In('Robot', self.destiny)]
        self.delete = [In('Robot', self.ubication)]
        self.effect = [In('Robot', self.destiny)]
    def action(self, state):
        if Holding('Key') in state:
            self.delete.append(In('Key', self.ubication))
            self.effect.append(In('Key', self.destiny))
        state = copy.deepcopy(state)
        if all(precondition in state for precondition in self.preconditions):
            for element in self.delete:
                if element in state:
                    state.remove(element)
            for element in self.effect:
                state.append(element)
        return list(set(state))
    def __str__(self):
        return f'Move from {self.ubication} to {self.destiny}'
        
    

In [7]:
def mutex(action1, action2):
    if any(elem in action1.delete for elem in action2.effect):
        return True
    if any(elem in action1.effect for elem in action2.delete):
        return True
    if any(elem in action1.delete for elem in action2.preconditions):
        return True
    if any(elem in action1.preconditions for elem in action2.delete):
        return True
    if any(~elem in action1.implications for elem in action2.preconditions):
        return True
    if any(~elem in action1.preconditions for elem in action2.implications):
        return True
    if any(elem in action1.effect for elem in action2.preconditions):
        return True
    if any(elem in action1.preconditions for elem in action2.effect):
        return True
    
    return False

In [8]:
def build_plan_graph(state, goal):
    grasp = GraspKey()
    lock = LockDoor()
    pk = PutKeyIntoBox()
    mv1 = Move('R2', 'R1')
    mv2 = Move('R1', 'R2')
    
    graph_levels = [state]
    
    i = 0
    while(True):
        # print(f'LEVEL {i+1}')
        new_level = [*graph_levels[i]]
        new_literals = grasp.action(graph_levels[i])
        for lit in new_literals:
            new_level.append(lit)
        new_literals = lock.action(graph_levels[i])
        # print(len(new_literals))
        for lit in new_literals:
            new_level.append(lit)
        new_literals = pk.action(graph_levels[i])
        for lit in new_literals:
            new_level.append(lit)
        new_literals = mv1.action(graph_levels[i])
        for lit in new_literals:
            new_level.append(lit)
        new_literals = mv2.action(graph_levels[i])
        for lit in new_literals:
            new_level.append(lit)
    
        graph_levels.append(list(set(new_level)))

        if all(elem in new_level for elem in goal):
            break
            
        if len(list(set(new_level))) == len(graph_levels[i]):
            return []
            break
        
        i += 1
            
    return graph_levels
        
        

In [9]:
def graph_heuristic(state, goal):
    graph = build_plan_graph(state, goal)
    if len(graph) > 0:
        return len(graph) - 1
    else:
        return float('inf')

In [10]:
grasp = GraspKey()
lock = LockDoor()
pk = PutKeyIntoBox()
mv1 = Move('R2', 'R1')
mv2 = Move('R1', 'R2')

In [11]:
ac = [grasp, lock, pk, mv1, mv2]

In [12]:
def get_possible_actions(action_list):
    not_mutex = []
    for i in range(len(action_list)):
        for j in range(i+1, len(action_list)):
            if not mutex(action_list[i], action_list[j]):
                # print('alg0')
                not_mutex.append([action_list[i], action_list[j]])
                
    for action in action_list:
        not_mutex.append([action])
    return not_mutex

In [13]:
actions = get_possible_actions(ac)

In [14]:
actions

[[<__main__.GraspKey at 0x221177f2160>],

In [15]:
class Node:
    def __init__(self, state, cost, heuristic, plan):
        self.state = state
        self.cost = cost
        self.heuristic = heuristic
        self.function = cost + heuristic
        self.plan = plan

In [16]:
node = Node(initial_state, 0, float('inf'), [])

In [17]:
def A_star(initial_node, goal, actions):
    frontier = [initial_node]
    while len(frontier) > 0:
        node = frontier.pop(0)
        # print('------------------------------------------')
        # print(' ^ '.join([str(x) for x in node.state]))
        # print('------------------------------------------')
        if all(elem in node.state for elem in goal):
            return node.plan
            break
        else:
            for action in actions:
                for sub_action in action:
                    # print(sub_action)
                    new_state = sub_action.action(node.state)
                    # print(' ^ '.join([str(x) for x in new_state]))
                   
                if all(elem in node.state for elem in new_state) and all(elem in new_state for elem in node.state) :
                    pass
                else:
                    h = graph_heuristic(new_state, goal)
                    c = node.cost + 1
                    # print(node.plan)
                    # print(node.plan.append(action))
                    new_node = Node(new_state, c, h, copy.deepcopy(node.plan))
                    new_node.plan.append(action)
                    # print(' ^ '.join([str(x) for x in new_state]))
                    # print(new_node.cost, new_node.heuristic)
                    # print(new_node.plan)
                    # node.plan.append(str(action))
                    # new_node.plan = node.plan
                    
                    frontier.append(new_node)
            
            frontier = sorted(frontier, key = lambda x: x.function)
    return 'NO HAY RESPUESTA'

In [18]:
node = Node(initial_state, 0, float('inf'), [])
goal = [Locked('Door'), In('Key', 'Box')]
plan = A_star(node, goal, actions)

In [19]:
plan = ' -> '.join([str(item) for sublist in plan for item in sublist])

In [20]:
plan

'Grasp Key -> Move from R2 to R1 -> Lock Door -> Put the key into the box'

## Dinner

In [42]:
class CleanHands:
    def __init__(self, negation = False):
        self.negation = negation
        
    def __hash__(self):
        return hash((self.negation))
        
    def __eq__(self, other):
        return isinstance(other, CleanHands) and self.negation == other.negation
    
    def __invert__(self):
        if self.negation:
             return CleanHands(negation = False)
        else:
            return CleanHands(negation = True)
    def __str__(self):
        if self.negation:
            return f'~CleanHands'
        else:   
            return f'CleanHands'
        
class Dinner:
    def __init__(self, negation = False):
        self.negation = negation
        
    def __hash__(self):
        return hash((self.negation))
        
    def __eq__(self, other):
        return isinstance(other, Dinner) and self.negation == other.negation
    
    def __invert__(self):
        if self.negation:
             return Dinner(negation = False)
        else:
            return Dinner(negation = True)
    def __str__(self):
        if self.negation:
            return f'~Dinner'
        else:
            return f'Dinner'
        
class Present:
    def __init__(self, negation = False):
        self.negation = negation
        
    def __hash__(self):
        return hash((self.negation))
        
    def __eq__(self, other):
        return isinstance(other, Present) and self.negation == other.negation
    
    def __invert__(self):
        if self.negation:
             return Present(negation = False)
        else:
            return Present(negation = True)
    def __str__(self):
        if self.negation:
            return f'~Present'
        else:
            return f'Present'
        
class Quiet:
    def __init__(self, negation = False):
        self.negation = negation
        
    def __hash__(self):
        return hash((self.negation))
        
    def __eq__(self, other):
        return isinstance(other, Quiet) and self.negation == other.negation
    
    def __invert__(self):
        if self.negation:
             return Quiet(negation = False)
        else:
            return Quiet(negation = True)
        
    def __str__(self):
        if self.negation:
            return f'~Quiet'
        else:
            return f'Quiet'
        
class Garbage:
    def __init__(self, negation = False):
        self.negation = negation
        
    def __hash__(self):
        return hash((self.negation))
        
    def __eq__(self, other):
        return isinstance(other, Garbage) and self.negation == other.negation
    
    def __invert__(self):
        if self.negation:
             return Garbage(negation = False)
        else:
            return Garbage(negation = True)
    def __str__(self):
        if self.negation:
            return f'~Garbage'
        else:
            return f'Garbage'
        

In [44]:
initial_state = [CleanHands(), Quiet(), Garbage()]
goal = [Dinner(), Present(), ~Garbage()]

In [49]:
class Cook:
    def __init__(self):
        self.preconditions = [CleanHands()]
        self.implications = [~CleanHands()]
        self.delete = []
        self.effect = [Dinner()]
    def action(self, state):
        state = copy.deepcopy(state)
        if all(precondition in state for precondition in self.preconditions):
            for element in self.delete:
                state.remove(element)
            for element in self.effect:
                state.append(element)
        return list(set(state))
    def __str__(self):
        return f'Cook'
    
class Wrap:
    def __init__(self):
        self.preconditions = [Quiet()]
        self.implications = [~Quiet()]
        self.delete = []
        self.effect = [Present()]
    def action(self, state):
        state = copy.deepcopy(state)
        if all(precondition in state for precondition in self.preconditions):
            for element in self.delete:
                state.remove(element)
            for element in self.effect:
                state.append(element)
        return list(set(state))
    def __str__(self):
        return f'Wrap Present'
    
    
class Carry:
    def __init__(self):
        self.preconditions = []
        self.implications = []
        self.delete = [Garbage(), CleanHands()]
        self.effect = [~Garbage(), ~CleanHands()]
    def action(self, state):
        state = copy.deepcopy(state)
        if all(precondition in state for precondition in self.preconditions):
            for element in self.delete:
                if element in state:
                    state.remove(element)
            for element in self.effect:
                state.append(element)
        return list(set(state))
    def __str__(self):
        return f'Carry garbage out'
    
class Dolly:
    def __init__(self):
        self.preconditions = []
        self.implications = []
        self.delete = [Garbage(), Quiet()]
        self.effect = [~Garbage(), ~Quiet()]
    def action(self, state):
        state = copy.deepcopy(state)
        if all(precondition in state for precondition in self.preconditions):
            for element in self.delete:
                if element in state:
                    state.remove(element)
            for element in self.effect:
                state.append(element)
        return list(set(state))
    def __str__(self):
        return f'Use dolly to take garbage out'

In [41]:
initial_state

In [55]:
dolly = Dolly()
wrap = Wrap()
cook = Cook()
carry = Carry()

In [51]:
print(' ^ '.join([str(x) for x in initial_state]))

CleanHands ^ Quiet ^ Garbage


In [52]:
new = dolly.action(initial_state)

In [88]:
new = wrap.action(initial_state)

In [89]:
print(' ^ '.join([str(x) for x in new]))

CleanHands ^ Quiet ^ Garbage ^ Present


In [58]:
mutex(cook, wrap)

False

In [59]:
ac = [dolly, wrap, cook, carry]

In [60]:
actions = get_possible_actions(ac)

In [65]:
print(' ^ '.join([str(x) for x in actions[3]]))

Wrap Present ^ Carry garbage out


In [61]:
actions

[[<__main__.Dolly at 0x22118a84070>, <__main__.Cook at 0x22119ba38e0>],
 [<__main__.Dolly at 0x22118a84070>, <__main__.Carry at 0x22119ba3580>],
 [<__main__.Wrap at 0x22119ba3df0>, <__main__.Cook at 0x22119ba38e0>],
 [<__main__.Wrap at 0x22119ba3df0>, <__main__.Carry at 0x22119ba3580>],

In [94]:

def build_plan_graph(state, goal, actions):    
    graph_levels = [state]
    
    i = 0
    while(True):
#         print(f'LEVEL {i+1}')
        new_level = [*graph_levels[i]]
#         new_literals = grasp.action(graph_levels[i])
#         print('STATE: ', ' ^ '.join([str(x) for x in graph_levels[i]]))
        for action in actions:
            for subaction in action:
#                 print(subaction)
                new_literals = subaction.action(graph_levels[i])
#             print('NEW STATE: ', ' ^ '.join([str(x) for x in new_literals]))
            for lit in new_literals:
                new_level.append(lit)
        
        graph_levels.append(list(set(new_level)))

        if all(elem in new_level for elem in goal):
            break
            
        if len(list(set(new_level))) == len(graph_levels[i]):
            return []
            break
        
        i += 1
    
            
    return graph_levels
        
        

In [95]:
build_plan_graph(initial_state, goal, actions)

[[<__main__.CleanHands at 0x22118a94970>,

In [101]:
def graph_heuristic(state, goal, actions):
    graph = build_plan_graph(state, goal, actions)
    if len(graph) > 0:
        return len(graph) - 1
    else:
        return float('inf')

In [113]:
def A_star(initial_node, goal, actions):
    frontier = [initial_node]
    while len(frontier) > 0:
        node = frontier.pop(0)
        print('------------------------------------------')
        print(' ^ '.join([str(x) for x in node.state]))
        print('------------------------------------------')
        if all(elem in node.state for elem in goal):
            return node.plan
            break
        else:
            for action in actions:
                aux_state = copy.deepcopy(node.state)
                print([str(x) for x in action])
                for sub_action in action:
                    
                    new_state = sub_action.action(aux_state)
                    aux_state = new_state
                
                print(' ^ '.join([str(x) for x in new_state]))
                   
                if all(elem in node.state for elem in new_state) and all(elem in new_state for elem in node.state) :
                    pass
                else:
                    h = graph_heuristic(new_state, goal, actions)
                    c = node.cost + 1
                    print(h, c)
#                     print(node.plan)
#                     print(node.plan.append(action))
                    new_node = Node(new_state, c, h, copy.deepcopy(node.plan))
                    new_node.plan.append(action)
                    # print(' ^ '.join([str(x) for x in new_state]))
                    # print(new_node.cost, new_node.heuristic)
                    # print(new_node.plan)
                    # node.plan.append(str(action))
                    # new_node.plan = node.plan
                    
                    frontier.append(new_node)
            
            frontier = sorted(frontier, key = lambda x: x.function)
    return 'NO HAY RESPUESTA'

In [114]:
node = Node(initial_state, 0, float('inf'), [])

In [115]:
plan = A_star(node, goal, actions)

------------------------------------------
CleanHands ^ Quiet ^ Garbage
------------------------------------------
['Use dolly to take garbage out', 'Cook']
CleanHands ^ ~Garbage ^ ~Quiet ^ Dinner
inf 1
['Use dolly to take garbage out', 'Carry garbage out']
~Garbage ^ ~Quiet ^ ~CleanHands
inf 1
['Wrap Present', 'Cook']
CleanHands ^ Quiet ^ Dinner ^ Garbage ^ Present
1 1
['Wrap Present', 'Carry garbage out']
Quiet ^ Present ^ ~Garbage ^ ~CleanHands
inf 1
['Use dolly to take garbage out']
CleanHands ^ ~Garbage ^ ~Quiet
inf 1
['Wrap Present']
CleanHands ^ Quiet ^ Garbage ^ Present
1 1
['Cook']
CleanHands ^ Quiet ^ Garbage ^ Dinner
1 1
['Carry garbage out']
Quiet ^ ~Garbage ^ ~CleanHands
inf 1
------------------------------------------
CleanHands ^ Quiet ^ Dinner ^ Garbage ^ Present
------------------------------------------
['Use dolly to take garbage out', 'Cook']
CleanHands ^ Dinner ^ ~Garbage ^ ~Quiet ^ Present
1 2
['Use dolly to take garbage out', 'Carry garbage out']
Dinner ^ ~Quiet 

In [116]:
plan = ' -> '.join([str(item) for sublist in plan for item in sublist])

In [117]:
plan

'Wrap Present -> Cook -> Use dolly to take garbage out -> Cook'